### **Produto cartesiano, preenchimento de dados vazios, exclusão de dados ausentes, discretização de atributos numéricos e preparação geral da base**

In [1190]:
import pandas as pd
import numpy as np

In [1191]:
df = pd.read_csv('..\\Bases\\Criados\\3_base_doencas_cardiacas.csv')

In [1192]:
df.columns[df.isnull().any()]

Index(['D00901', 'D014', 'E006011', 'E008', 'E01602', 'E017', 'E01802', 'E019',
       'F001021', 'F007021', 'F008021', 'VDF00102', 'G060', 'G062', 'I006',
       'I004', 'I00401', 'I00402', 'I00403', 'J00402', 'J00404', 'J00801',
       'M00203', 'M011011', 'M011021', 'M011051', 'M011071', 'N004', 'N005',
       'N008', 'P00104', 'P00404', 'P02801', 'P029', 'P035', 'P03701',
       'P03702', 'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407',
       'P05410', 'P05413', 'P05416', 'P05419', 'Q00201', 'Q03001', 'Q060',
       'R034', 'H003', 'H004', 'H010', 'D00301', 'D008', 'E002', 'E003',
       'E004', 'E005', 'E011', 'E014011', 'E01601', 'E01603', 'E01605',
       'E01801', 'I00103', 'I001031', 'P00103', 'P00403', 'P05901', 'P05902',
       'P05903', 'P05904', 'H002'],
      dtype='object')

In [1193]:
(df.isnull().sum() > 0).sum()

np.int64(75)

### C009 - Cor ou raça

In [1194]:
# Filtrar os registros onde C009 == 9.0
filtro_cor_9 = df[df['C009'] == 9.0]

# Exibir apenas as colunas C009 e Q06306 dessas instâncias
print(filtro_cor_9[['C009', 'Q06306']])

       C009  Q06306
99      9.0     2.0
10621   9.0     2.0


In [1195]:
print(df['C009'].value_counts(dropna=False))

C009
4.0    11677
1.0     5933
2.0     2619
5.0      163
3.0      134
9.0        2
Name: count, dtype: int64


In [1196]:
# Remover as instâncias onde C009 é igual a 9.0
df = df[df['C009'] != 9.0]
print(df['C009'].value_counts()) #conferindo

C009
4.0    11677
1.0     5933
2.0     2619
5.0      163
3.0      134
Name: count, dtype: int64


In [1197]:
import pandas as pd

df['C009'] = df['C009'].map({
    1.0: 'Branca',
    2.0: 'Preta',
    4.0: 'Parda'
})

# Preencher os valores restantes com 'Outra'
df['C009'] = df['C009'].fillna('Outra')

print(df['C009'].value_counts())

C009
Parda     11677
Branca     5933
Preta      2619
Outra       297
Name: count, dtype: int64


### Escolaridade

Atributos utilizados:
1. D00201: se frequenta escola atualmente
2. D00301: qual curso frequenta
3. D008: se já frequentou escola ou creche
4. D00901: Qual foi o curso mais elevado que frequentou?
5. D014: se concluiu esse curso

In [1198]:
def categorizar_escolaridade(row):
    if row['D00201'] == 1:
        return 'Frequenta'
    elif row['D00201'] == 2:
        if row['D008'] == 1:
            return 'Frequentou'
        elif row['D008'] == 2:
            return 'Não frequentou'
    return pd.NA  # Caso não se encaixe em nenhuma condição

df['Situacao_escolar'] = df.apply(categorizar_escolaridade, axis=1)

print(df['Situacao_escolar'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Situacao_escolar'].isna().sum())


Situacao_escolar
Frequentou        18137
Não frequentou     1748
Frequenta           641
Name: count, dtype: int64
Valores faltantes: 0


In [1199]:
df['D00301'].value_counts()

D00301
8.0     213
9.0     113
4.0      98
5.0      88
6.0      42
3.0      38
7.0      22
10.0     19
11.0      8
Name: count, dtype: int64

In [1200]:
def map_escolaridade(row): 
    situacao = row['Situacao_escolar']
    curso = row['D00901']
    concluiu = row['D014']
    curso_atual = row['D00301']

    # Se nunca frequentou escola, categoria = 0
    if situacao == 'Não frequentou':
        return 0

    # Usa D00901 (curso mais elevado) prioritariamente
    if curso in [1, 2]:  # Creche, Pré-escola
        categoria = 1
    elif curso in [3, 4, 5, 6, 7, 8]:  # Alfabetização, primário, EJA fundamental etc.
        categoria = 2
    elif curso in [9, 10, 11]:  # Ensino médio, EJA médio
        categoria = 3
    elif curso == 12:  # Superior
        categoria = 4
    elif curso in [13, 14, 15]:  # Pós-graduação
        categoria = 5
    # Se curso é inválido ou ausente, tenta usar curso_atual (D00301)
    elif curso_atual in [3, 4, 5]:
        categoria = 1  # Educação infantil
    elif curso_atual in [6, 7]:
        categoria = 2  # Fundamental
    elif curso_atual in [8, 9]:
        categoria = 3  # Médio
    elif curso_atual in [10, 11]:
        categoria = 4  # Superior
    else:
        return np.nan

    # Rebaixar se não concluiu (exceto se usou curso_atual)
    if concluiu == 2 and curso in range(1, 16):
        categoria -= 1
        if categoria < 0:
            categoria = 0

    return categoria



df['Escolaridade'] = df.apply(map_escolaridade, axis=1)

print(df['Escolaridade'].value_counts(dropna=False))

# 0: Nunca frequentou escola
# 1: Educação infantil
# 2: Ensino fundamental
# 3: Ensino médio
# 4: Ensino superior
# 5: Pós-graduação

Escolaridade
1    7855
3    4920
2    3456
0    1748
4    1669
5     878
Name: count, dtype: int64


In [1201]:
df.drop(columns=['D00201', 'D00301', 'D008', 'D00901', 'D014', 'Situacao_escolar'], inplace=True)

### Renda

1. E014011: se era trabalhador não remunerado ajudando parente/domicílio
2. E01601: se recebia dinheiro no trabalho principal
3. E01602: valor da renda do trabalho principal
4. E01801: se recebia dinheiro em outro(s) trabalho(s)
5. E01802: valor da renda dos outros trabalhos

Regras:

1. Se E01601 != 1 ou E01801 != 1, a renda respectiva é 0;
2. Se a pessoa é não remunerada (E014011 = 1, 2, 3), renda é 0;
3. Se E01602 ou E01802 estiverem NaN, mas a pessoa disse que recebia (E01601 ou E01801 = 1), considerar como NaN final.
4. Renda total é a soma de E01602 e E01802.

In [1202]:
import pandas as pd
import numpy as np

def calcular_renda(row):
    # Regra 2: trabalhador não remunerado (1, 2 ou 3)
    if row['E014011'] in [1, 2, 3]:
        return 0.0

    # Regra 1 + 3: rendimentos do trabalho
    renda1 = row['E01602'] if row['E01601'] == 1 and pd.notna(row['E01602']) else 0.0
    renda2 = row['E01802'] if row['E01801'] == 1 and pd.notna(row['E01802']) else 0.0

    # Outros rendimentos
    renda3 = row['F001021'] if row['F001011'] == 1 and pd.notna(row['F001021']) else 0.0
    renda4 = row['F007021'] if row['F007011'] == 1 and pd.notna(row['F007021']) else 0.0
    renda5 = row['F008021'] if row['F008011'] == 1 and pd.notna(row['F008021']) else 0.0
    renda6 = row['VDF00102'] if row['VDF001'] == 1 and pd.notna(row['VDF00102']) else 0.0

    # Se todos forem NaN, retorna NaN
    rendas = [renda1, renda2, renda3, renda4, renda5, renda6]
    if all(pd.isna(r) for r in rendas):
        return np.nan

    return sum(r for r in rendas if pd.notna(r))

# Aplicando ao DataFrame
df['Renda'] = df.apply(calcular_renda, axis=1)

print("Valores faltantes:", df['Renda'].isna().sum())


Valores faltantes: 0


In [1203]:
print((df['Renda'] == 0.0).sum())

2522


In [1204]:
df.drop(columns=['E014011', 'E01601', 'E01801', 'F001011', 'F007011', 'F008011', 'VDF001', 'E01602', 'E01802', 'F001021', 'F007021', 'F008021', 'VDF00102'], inplace=True)

#### Discretização do atributo Renda

In [1205]:
def faixa_renda_sm(renda):
    if renda == 0:
        return 'Sem renda'
    elif renda <= 998:
        return 'Até 1 SM'
    elif renda <= 2 * 998:
        return '1–2 SM'
    elif renda <= 3 * 998:
        return '2–3 SM'
    elif renda <= 5 * 998:
        return '3–5 SM'
    else:
        return 'Mais de 5 SM'

df['Renda'] = df['Renda'].apply(faixa_renda_sm)
print(df['Renda'].value_counts(dropna=False))


Renda
Até 1 SM        8432
1–2 SM          4548
Sem renda       2522
2–3 SM          1852
Mais de 5 SM    1722
3–5 SM          1450
Name: count, dtype: int64


In [1206]:
mapeamento_faixas = {
    'Sem renda': 0,
    'Até 1 SM': 1,
    '1–2 SM': 2,
    '2–3 SM': 3,
    '3–5 SM': 4,
    'Mais de 5 SM': 5
}

df['Renda'] = df['Renda'].map(mapeamento_faixas)
print(df['Renda'].value_counts(dropna=False))

Renda
1    8432
2    4548
0    2522
3    1852
5    1722
4    1450
Name: count, dtype: int64


### Horas trabalhadas *incompleto (checar quant trabalho 3. Três ou mais)

1. E011: número de trabalhos na semana 
2. E017: horas trabalhadas em 1 trabalho
3. E019: horas trabalhadas em mais de 1 trabalho

In [1207]:
def quantidade_trabalhos(row):
    if row['E011'] == 1:
        return 1
    elif row['E011'] == 2:
            return 2
    elif row['E011'] == 3:
            return 3
    else:
         return 0

df['Quant_trabalhos'] = df.apply(quantidade_trabalhos, axis=1)

print(df['Quant_trabalhos'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Quant_trabalhos'].isna().sum())

Quant_trabalhos
1    11210
0     8592
2      642
3       82
Name: count, dtype: int64
Valores faltantes: 0


In [1208]:
print("E017 faltantes:", df['E017'].isna().sum())
print("E019 faltantes:", df['E019'].isna().sum())

E017 faltantes: 8592
E019 faltantes: 19802


In [1209]:
df['Quant_trabalhos'] = df['Quant_trabalhos'].astype(int)
df.loc[df['Quant_trabalhos'] == 0, 'E017'] = 0
df.loc[df['Quant_trabalhos'].isin([0, 1]), 'E019'] = 0

#df['E017']=df['E017'].fillna(0)
#df['E019']=df['E019'].fillna(0)

In [1210]:
print("E017 faltantes:", df['E017'].isna().sum())
print("E019 faltantes:", df['E019'].isna().sum())

E017 faltantes: 0
E019 faltantes: 0


In [1211]:
df['Carga_horaria_semanal'] = df['E017'] + df['E019']

print("Valores faltantes:", df['Carga_horaria_semanal'].isna().sum())

Valores faltantes: 0


In [1212]:
df['Quant_trabalhos'] = pd.to_numeric(df['Quant_trabalhos'], errors='coerce')
df[(df['Renda'] == 0.0) & (df['Quant_trabalhos'] == 0) & (df['Carga_horaria_semanal'] == 0.0)][['Renda', 'Quant_trabalhos', 'Carga_horaria_semanal']]

,Renda,Quant_trabalhos,Carga_horaria_semanal
3,0,0,0.0
5,0,0,0.0
24,0,0,0.0
33,0,0,0.0
35,0,0,0.0
...,...,...,...
20470,0,0,0.0
20473,0,0,0.0
20474,0,0,0.0
20475,0,0,0.0


#### Discretização do atributo Carga horária semanal

In [1213]:
def categorizar_carga_horaria(horas):
    if horas == 0:
        return 'Sem carga horária'
    elif horas <= 44:
        return 'Até 44h'
    else:
        return 'Acima de 44h'

df['Carga_horaria_semanal'] = df['Carga_horaria_semanal'].apply(categorizar_carga_horaria)
print(df['Carga_horaria_semanal'].value_counts(dropna=False))

Carga_horaria_semanal
Até 44h              9296
Sem carga horária    8592
Acima de 44h         2638
Name: count, dtype: int64


In [1214]:
mapeamento = {
    'Sem carga horária': 0,
    'Até 44h': 1,
    'Acima de 44h': 2
}

df['Carga_horaria_semanal'] = df['Carga_horaria_semanal'].map(mapeamento)
print(df['Carga_horaria_semanal'].value_counts(dropna=False))

Carga_horaria_semanal
1    9296
0    8592
2    2638
Name: count, dtype: int64


In [1215]:
df.drop(columns=['E01603', 'E01605', 'E017', 'E019'], inplace=True)

### Afastamento do trabalho por motivo de saúde & motivo de saúde relacionado ao trabalho (eliminar)

1. E005: tinha algum trabalho remunerado do qual estava temporariamente afastado?
2. E006011: por que motivo estava afastado desse trabalho
3. E008: a doença ou acidente foi relacionado ao trabalho?

In [1216]:
def temporariamente_afastado(row):
    if row['E005'] == 1:
        return 'Sim'
    elif row['E005'] == 2:
            return 'Não'
    else:
         return 'Não'

df['Temporariamente_afastado'] = df.apply(temporariamente_afastado, axis=1)


print(df['Temporariamente_afastado'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Temporariamente_afastado'].isna().sum())

Temporariamente_afastado
Não    20201
Sim      325
Name: count, dtype: int64
Valores faltantes: 0


In [1217]:
df.loc[(df['Temporariamente_afastado']== 'Não'), 'E006011'] = 0 #se E006011=0, não esteve afastado

In [1218]:
df['E006011'].isna().sum()

np.int64(0)

In [1219]:
df.loc[(df['E006011'].isin([0, 1, 2, 4, 5, 6, 7])), 'E008'] = 0

In [1220]:
df['E008'].isna().sum()

np.int64(0)

In [1221]:
pd.set_option('future.no_silent_downcasting', True)

df['Temporariamente_afastado'] = df['Temporariamente_afastado'].replace({
    'Sim': 1,
    'Não': 2
}).astype(float)

In [1222]:
df.drop(columns=['E001','E002', 'E003', 'E004', 'E005'], inplace=True)

### Mobilidade

1. G059: usa algum aparelho de auxílio para se locomover?
2. G060: faz uso de cadeira de rodas?
3. G062: faz uso de bengala, muletas ou andador?

In [1223]:
def classificar_mobilidade(row):
    usa_aparelho = row['G059'] == 1
    cadeira_rodas = row['G060'] == 1
    bengala_muleta = row['G062'] == 1

    if not usa_aparelho:
        return 'Não usa aparelho de auxílio'
    if cadeira_rodas:
        return 'Apenas cadeira de rodas'
    if bengala_muleta:
        return 'Apenas bengala/muleta/andador'
    if cadeira_rodas and bengala_muleta:
        return 'Usa cadeira de rodas e bengala/muleta/andador'
    if not cadeira_rodas and not bengala_muleta:
        return 'Usa outros aparelhos de auxílio'
    return np.nan  # Caso de inconsistência ou valores faltantes

df['Mobilidade_assistida'] = df.apply(classificar_mobilidade, axis=1)

# Visualizar resultado:
print(df['Mobilidade_assistida'].value_counts(dropna=False))
print(df['Mobilidade_assistida'].isna().sum())

Mobilidade_assistida
Não usa aparelho de auxílio        20178
Apenas bengala/muleta/andador        239
Apenas cadeira de rodas               75
Usa outros aparelhos de auxílio       34
Name: count, dtype: int64
0


In [1224]:
df = df.drop(columns=['G060', 'G062'])

### Plano de Saúde

1. I00102: se tem plano
2. I006: avaliação do plano

In [1225]:
def classificar_plano(row):
    tem_plano = row['I00102']
    avaliacao = row['I006']
    
    if tem_plano == 2:
        return 'Não tem plano'  # Não tem plano
    elif avaliacao in [1, 2]:
        return 'Bom/muito bom'  # Muito bom ou bom
    elif avaliacao == 3:
        return 'Regular'  # Regular
    elif avaliacao in [4, 5]:
        return 'Ruim/muito ruim' # Ruim ou muito ruim
    elif avaliacao == 6:
        return 'Nunca usou'  # Nunca usou
    else:
        return np.nan  # Valores faltantes ou inválidos

df['Avaliacao_plano'] = df.apply(classificar_plano, axis=1)

print(df['Avaliacao_plano'].value_counts(dropna=False))
print(df['Avaliacao_plano'].isna().sum())

# 0     | Não tem plano                
# 1     | Avaliação muito boa ou boa   
# 2     | Avaliação regular            
# 3     | Avaliação ruim ou muito ruim 
# 4     | Nunca usou o plano 

Avaliacao_plano
Não tem plano      16685
Bom/muito bom       2822
Regular              794
Ruim/muito ruim      141
Nunca usou            84
Name: count, dtype: int64
0


In [1226]:
df[['Avaliacao_plano']]

,Avaliacao_plano
0,Não tem plano
1,Regular
2,Regular
3,Não tem plano
4,Não tem plano
...,...
20523,Não tem plano
20524,Não tem plano
20525,Não tem plano
20526,Não tem plano


In [1227]:
# Mapeamento explícito
mapeamento = {
    "Não tem plano": 0,
    "Bom/muito bom": 1,
    "Regular": 2,
    "Ruim/muito ruim": 3,
    "Nunca usou": 4
}

# Aplicando o mapeamento
df["Avaliacao_plano"] = df["Avaliacao_plano"].replace(mapeamento).astype(int)

In [1228]:
df[["Avaliacao_plano"]]

,Avaliacao_plano
0,0
1,2
2,2
3,0
4,0
...,...
20523,0
20524,0
20525,0
20526,0


In [1229]:
df = df.drop(columns=['I006'])

3. I004: se é de instituição pública
4. I00103: se é morador 
5. I001031: número do titular

*C00301 == I001031 --> é titular    

In [1230]:
def definir_Instituicao_plano(row):
    if row['I00102'] != 1:
        return 0.0  # Não tem plano
    if row['I00103'] != 1 and row['C00301'] != row['I001031']: # não é morador
        return 0.0  # não é titular do plano
    if row['I004'] == 1:
        return 1.0  # Plano é de instituição pública
    if row['I004'] == 2:
        return 2.0  # Plano é de instituição privada
    
    return 0.0

df['Instituicao_plano'] = df.apply(definir_Instituicao_plano, axis=1)

print(df['Instituicao_plano'].value_counts(dropna=False))
print(df['Instituicao_plano'].isna().sum())

#0     | Não tem plano e/ou não é morador + titular do plano              
#1     | Plano é de instituição pública   
#2     | Plano é de instituição privada   

Instituicao_plano
0.0    17654
2.0     1814
1.0     1058
Name: count, dtype: int64
0


In [1231]:
df = df.drop(columns=['I004'])

6. I00401: plano dá direito a consultas
7. I00402: plano dá direito a exames
8. I00403: plano dá direito a internações

In [1232]:
def Cobertura_plano(row):
    # Se não tem plano, retorna 0.0
    if pd.isna(row['I00102']) or row['I00102'] != 1:
        return 0.0

    # Verifica titularidade
    if pd.isna(row['I00103']) or row['I00103'] != 1:
        return 0.0
    if pd.isna(row['C00301']) or pd.isna(row['I001031']) or row['C00301'] != row['I001031']:
        return 0.0

    # Verifica cobertura
    coberturas = [
        row['I00401'] == 1 if pd.notna(row['I00401']) else False,
        row['I00402'] == 1 if pd.notna(row['I00402']) else False,
        row['I00403'] == 1 if pd.notna(row['I00403']) else False
    ]
    total_cobertura = sum(coberturas)

    if total_cobertura == 1:
        return 1.0
    elif total_cobertura == 2:
        return 2.0
    elif total_cobertura == 3:
        return 3.0
    elif total_cobertura == 0:
        return 4.0  # Não cobre nada, apesar de ter plano e titular
    else:
        return np.nan  # Caso de erro

df['Cobertura_plano'] = df.apply(Cobertura_plano, axis=1)

print(df['Cobertura_plano'].value_counts(dropna=False))
print("Valores nulos:", df['Cobertura_plano'].isna().sum())


# 1- se o plano cobre uma das coisas
# 2- se o plano cobre duas das coisas
# 3- se o plano cobre as três coisas

Cobertura_plano
0.0    17654
3.0     2677
2.0      151
1.0       29
4.0       15
Name: count, dtype: int64
Valores nulos: 0


In [1233]:
df = df.drop(columns=['I00102', 'I00401', 'I00402', 'I00403', 'I00103', 'I001031'])

### Preenchendo aqui os atributos J00402, J00404 e J00801 (eliminar)

In [1234]:
df.loc[df['J002'] == 2, 'J00402'] = 0
df.loc[df['J002'] == 2, 'J00404'] = 0

In [1235]:
print(df['J00402'].isna().sum())
print(df['J00404'].isna().sum())

0
0


In [1236]:
df.loc[df['J007'] == 2, 'J00801'] = 0
print(df['J00801'].isna().sum())

0


### Exposição tóxica no trabalho

Atributos:
1. M011011: substâncias químicas
2. M011021: barulho intenso (ruído)
3. M011051: lixo (resíduo urbano)
4. M011071: poeira mineral

In [1237]:
def marcar_exposicao_quimica(row):
    trabalho = row['E011']
    exposicao = row['M011011']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_quimica'] = df.apply(marcar_exposicao_quimica, axis=1)

print(df['Exposicao_quimica'].value_counts(dropna=False))
print(df['Exposicao_quimica'].isna().sum())

Exposicao_quimica
2    19154
1     1372
Name: count, dtype: int64
0


In [1238]:
def marcar_exposicao_ruido(row):
    trabalho = row['E011']
    exposicao = row['M011021']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_ruido'] = df.apply(marcar_exposicao_ruido, axis=1)

print(df['Exposicao_ruido'].value_counts(dropna=False))
print(df['Exposicao_ruido'].isna().sum())

Exposicao_ruido
2    18313
1     2213
Name: count, dtype: int64
0


In [1239]:
def marcar_exposicao_lixo(row):
    trabalho = row['E011']
    exposicao = row['M011051']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_lixo'] = df.apply(marcar_exposicao_lixo, axis=1)

print(df['Exposicao_lixo'].value_counts(dropna=False))
print(df['Exposicao_lixo'].isna().sum())

Exposicao_lixo
2    19756
1      770
Name: count, dtype: int64
0


In [1240]:
def marcar_exposicao_minerio(row):
    trabalho = row['E011']
    exposicao = row['M011071']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_minerio'] = df.apply(marcar_exposicao_minerio, axis=1)

print(df['Exposicao_minerio'].value_counts(dropna=False))
print(df['Exposicao_minerio'].isna().sum())

Exposicao_minerio
2    19191
1     1335
Name: count, dtype: int64
0


In [1241]:
df = df.drop(columns=['M011011'])
df = df.drop(columns=['M011021'])
df = df.drop(columns=['M011051'])
df = df.drop(columns=['M011071'])
df = df.drop(columns=['E011'])

### Dor/desconforto no peito

1. N004: dor ou desconforto no peito ao subir uma ladeira, um lance de escadas ou caminhar rápido no plano
2. N005: dor ou desconforto no peito ao caminhar em lugar plano
3. N008: onde é a dor/desconforto no peito

Outros atributos usados: 

4. G059: usa algum aparelho de auxílio para se locomover? (Se G59=1 , siga N10) - incompleto!

In [1242]:
def map_dor_peito(row):
    if row['G059'] != 2:
        return 'Não sente dor'  # Ignora quem não respondeu 2 em G059 (736 pessoas)

    if row['N004'] == 2 and row['N005'] == 2:
        return 'Não sente dor'

    if row['N004'] == 1 or row['N005'] == 1:
        dor_map = {
            1: 'Acima ou meio do peito',
            2: 'Abaixo do peito',
            3: 'Braço esquerdo',
            4: 'Outro lugar'
        }
        return dor_map.get(row['N008'], pd.NA)

    return pd.NA  # Se não cair em nenhum caso anterior 

df['Dor_peito'] = df.apply(map_dor_peito, axis=1)

print(df['Dor_peito'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Dor_peito'].isna().sum()}")

Dor_peito
Não sente dor             17642
Acima ou meio do peito     2225
Abaixo do peito             418
Braço esquerdo              199
Outro lugar                  42
Name: count, dtype: int64
Valores faltantes: 0


In [1243]:
df = df.drop(columns=['G059', 'N004', 'N005', 'N008'])

### Exercício físico 

1. P034: Nos últimos três meses, praticou algum tipo de exercício físico ou esporte?
2. P035: Quantos dias por semana.
3. P03701: tempo da sessão em horas.
4. P03702: tempo da sessão em minutos.

In [1244]:
print(df['P035'].max())
print(df['P03701'].max())

7.0
10.0


In [1245]:
df['P034'].value_counts()

P034
2.0    13135
1.0     7391
Name: count, dtype: int64

In [1246]:
# Função para calcular tempo semanal em minutos
def calcular_minutos_exercicio(row):
    if row['P034'] == 2:
        return 0
    elif row['P034'] == 1 and row['P035'] > 0:
        horas = row['P03701'] if pd.notnull(row['P03701']) else 0
        minutos = row['P03702'] if pd.notnull(row['P03702']) else 0
        tempo_semanal_min = row['P035'] * (horas * 60 + minutos)
        return min(max(tempo_semanal_min, 0), 420)  # 420 min = 7h como limite superior opcional
    else:
        return 0

# Aplica o cálculo ao DataFrame
df['Atividade_fisica_semanal'] = df.apply(calcular_minutos_exercicio, axis=1)

# Função para categorizar com base na OMS
def categorizar_atividade_fisica(minutos):
    if minutos == 0:
        return 0  # Sedentário
    elif 0 < minutos < 150:
        return 1  # Insuficiente
    elif 150 <= minutos <= 300:
        return 2  # Ideal
    elif minutos > 300:
        return 3  # Acima do ideal
    else:
        return np.nan

# Aplica a categorização
df['Atividade_fisica_semanal'] = df['Atividade_fisica_semanal'].apply(categorizar_atividade_fisica)

# Resultados
print(df['Atividade_fisica_semanal'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Atividade_fisica_semanal'].isna().sum()}")


Atividade_fisica_semanal
0    13325
2     3287
1     2342
3     1572
Name: count, dtype: int64
Valores faltantes: 0


In [1247]:
df[['Atividade_fisica_semanal']]

,Atividade_fisica_semanal
0,0
1,2
2,0
3,3
4,2
...,...
20523,0
20524,0
20525,0
20526,0


In [1248]:
df = df.drop(columns=['P034', 'P035', 'P03701', 'P03702'])

### Hábito de fumar / tabagismo

1. P050: Atualmente, o(a) Sr(a) fuma algum produto do tabaco? 
2. P051: E no passado, o(a) Sr(a) fumou algum produto do tabaco diariamente?
3. P052: E no passado, o(a) Sr(a) fumou algum produto do tabaco?
4. P053:  Que idade o(a) Sr(a) tinha quando começou a fumar produto do tabaco diariamente?

    P050 (Fuma atualmente)

1 = Sim, diariamente → usa P053

2 = Sim, menos que diariamente → verifica P051

3 = Não fuma atualmente → verifica P052

    P051 (Fumou diariamente no passado)

1 = Sim → usa P053

2 = Não → considera como não fumante diário

    P052 (Já fumou alguma vez)

1 = Sim → usa P053

2 ou 3 = Não fumou

In [1249]:
def categorizar_inicio_fumo(row):
    p50 = row['P050']
    p51 = row['P051']
    p52 = row['P052']
    p53 = row['P053']  # idade de início

    # Fuma diariamente atualmente
    if p50 == 1:
        idade = p53

    # Fuma menos que diariamente → verifica se já fumou diariamente
    elif p50 == 2:
        if p51 == 1:
            idade = p53
        else:
            return 'Não fuma' 

    # Não fuma atualmente → verifica se já fumou algum dia
    elif p50 == 3:
        if p52 == 1:
            idade = p53
        else:
            return 'Não fuma'

    else:
        return 'Sem informação'  # Resposta inválida ou faltante

    # Categoriza com base na idade de início
    if pd.notna(idade):
        if idade < 18:
            return 'Início antes dos 18'
        elif 18 <= idade <= 30:
            return 'Início entre 18 e 30'
        elif idade > 30:
            return 'Início após os 30'
    return 'Sem informação'  # idade ausente

df['Inicio_tabagismo'] = df.apply(categorizar_inicio_fumo, axis=1)

print(df['Inicio_tabagismo'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Inicio_tabagismo'].isna().sum()}")


Inicio_tabagismo
Não fuma                12101
Início antes dos 18      5458
Início entre 18 e 30     2695
Início após os 30         272
Name: count, dtype: int64
Valores faltantes: 0


In [1250]:
df[["Inicio_tabagismo"]]

,Inicio_tabagismo
0,Não fuma
1,Não fuma
2,Início antes dos 18
3,Não fuma
4,Não fuma
...,...
20523,Não fuma
20524,Não fuma
20525,Início entre 18 e 30
20526,Início após os 30


In [1251]:
# Mapeamento explícito
mapeamento = {
    "Não fuma": 0,
    "Início antes dos 18": 1,
    "Início entre 18 e 30": 2,
    "Início após os 30": 3
}

# Aplicando o mapeamento
df["Inicio_tabagismo"] = df["Inicio_tabagismo"].replace(mapeamento).astype(int)

In [1252]:
df[["Inicio_tabagismo"]]

,Inicio_tabagismo
0,0
1,0
2,1
3,0
4,0
...,...
20523,0
20524,0
20525,2
20526,3


 P59. Há quanto tempo o(a) Sr(a) parou de fumar?

5. P05901 - Anos
6. P05902 - Meses
7. P05903 - Semanas
8. P05904 - Dias

In [1253]:
def tempo_parou_fumar(row):
    # Fumante atual
    if row['P050'] in [1, 2]:
        return 'Fumante atual'

    # Nunca fumou
    if row['P052'] == 3:
        return 'Nunca fumou'

    # Ex-fumante: calcular tempo total em meses
    if row['P052'] in [1, 2]:
        anos = row.get('P05901', 0) or 0
        meses = row.get('P05902', 0) or 0
        semanas = row.get('P05903', 0) or 0
        dias = row.get('P05904', 0) or 0

        total_meses = anos * 12 + meses + (semanas / 4.345) + (dias / 30)

        if pd.isna(total_meses):
            return pd.NA

        if total_meses < 60:
            return 'Parou há <5 anos'
        else:
            return 'Parou há >5 anos'

    return pd.NA


df['Tempo_parou_fumar'] = df.apply(tempo_parou_fumar, axis=1)

print(df['Tempo_parou_fumar'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Tempo_parou_fumar'].isna().sum()}")

Tempo_parou_fumar
Nunca fumou         11265
Parou há >5 anos     5493
Fumante atual        2862
Parou há <5 anos      906
Name: count, dtype: int64
Valores faltantes: 0


In [1254]:
df[["Tempo_parou_fumar"]]

,Tempo_parou_fumar
0,Nunca fumou
1,Parou há >5 anos
2,Parou há >5 anos
3,Nunca fumou
4,Nunca fumou
...,...
20523,Nunca fumou
20524,Nunca fumou
20525,Fumante atual
20526,Fumante atual


In [1255]:
# Mapeamento explícito
mapeamento = {
    "Nunca fumou": 0,
    "Fumante atual": 1,
    "Parou há <5 anos": 2,
    "Parou há >5 anos": 3
}

# Aplicando o mapeamento
df["Tempo_parou_fumar"] = df["Tempo_parou_fumar"].replace(mapeamento).astype(int)

In [1256]:
df[["Tempo_parou_fumar"]]

,Tempo_parou_fumar
0,0
1,3
2,3
3,0
4,0
...,...
20523,0
20524,0
20525,1
20526,1


9. Cigarros industrializados (P05401)
10. Cigarros de palha ou enrolados (P05404)
11. Cigarros de cravo ou Bali (P05407)
12. Cachimbo (P05410)
13. Charutos ou cigarrilhas (P05413)
14. Narguilé (P05416)
15. Outro (P05419)

In [1257]:
import pandas as pd

# Dicionário de pesos baseados na frequência semanal estimada
pesos_fumo = {
    1: 7.0,   # Um ou mais por dia
    2: 3.0,   # Um ou mais por semana
    3: 1.0,   # Menos que uma vez por semana
    4: 0.3,   # Menos do que um por mês
    5: 0.0    # Não fuma este produto
}

# Atributos de produtos fumados
colunas_fumo = [
    'P05401',  # Cigarros industrializados
    'P05404',  # Cigarros de palha
    'P05407',  # Cigarros de cravo
    'P05410',  # Cachimbo
    'P05413',  # Charuto
    'P05416',  # Narguilé
    'P05419'   # Outro
]

# Substituir as categorias pelos pesos estimados
for col in colunas_fumo:
    df[col + '_peso'] = df[col].map(pesos_fumo)

# Calcular a intensidade total somando os pesos
df['Intensidade_fumo'] = df[[col + '_peso' for col in colunas_fumo]].sum(axis=1).round(1)

# Discretização com base na intensidade total semanal
def classificar_fumante(valor):
    if valor == 0.0:
        return 'Não fumante'
    elif valor <= 1.0:
        return 'Fumante esporádico'
    elif valor <= 7.0:
        return 'Fumante moderado'
    else:
        return 'Fumante intenso'

df['Intensidade_fumo'] = df['Intensidade_fumo'].apply(classificar_fumante)

print(df['Intensidade_fumo'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Intensidade_fumo'].isna().sum()}")


Intensidade_fumo
Não fumante           17664
Fumante moderado       2410
Fumante intenso         383
Fumante esporádico       69
Name: count, dtype: int64
Valores faltantes: 0


In [1258]:
# Mapeamento explícito
mapeamento = {
    "Não fumante": 0,
    "Fumante esporádico": 1,
    "Fumante moderado": 2,
    "Fumante intenso": 3
}

# Aplicando o mapeamento
df["Intensidade_fumo"] = df["Intensidade_fumo"].replace(mapeamento).astype(int)
print(df['Intensidade_fumo'].value_counts(dropna=False))

Intensidade_fumo
0    17664
2     2410
3      383
1       69
Name: count, dtype: int64


In [1259]:
df = df.drop(columns=['P050', 'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419', 'P05901', 'P05902', 'P05903', 'P05904', 
                      'P05401_peso', 'P05404_peso', 'P05407_peso', 'P05410_peso', 'P05413_peso', 'P05416_peso', 'P05419_peso'])

### Atributos do módulo Q (diagnósticos)

In [1260]:
df.loc[df['Q00101'] == 6, 'Q00201'] = 2 #diagnóstico de hipertensão
df.loc[df['Q02901'] == 6, 'Q03001'] = 2 #diagnóstico de diabetes
df.loc[df['Q05901'] == 6, 'Q060'] = 2 #diagnóstico de colesterol alto

In [1261]:
print(df['Q00201'].isna().sum())
print(df['Q03001'].isna().sum())
print(df['Q060'].isna().sum())

0
0
0


In [1262]:
df = df.drop(columns=['Q00101', 'Q02901', 'Q05901'])    

### Preenchendo aqui os atributos H003, H004 e H010

In [1263]:
df.loc[df['H001'].isin([4, 5]), 'H002'] = 0

In [1264]:
print(df['H002'].isna().sum())

0


In [1265]:
df.loc[(df['H002'].isin([1, 0])), 'H003'] = 0

In [1266]:
print(df['H003'].isna().sum())

0


In [1267]:
df.loc[df['H003'] == 0, 'H004'] = 0

In [1268]:
print(df['H004'].isna().sum())

0


In [1269]:
df.loc[df['H004'] != 1, 'H010'] = 0

In [1270]:
print(df['H010'].isna().sum())

0


### Hábito de beber / alcoolismo - incompleto (só checar)

1. P027: Com que frequência costuma consumir alguma bebida alcoólica?
2. P02801: Quantos dias por semana costuma consumir alguma bebida alcoólica?
3. P029:  Em geral, no dia que bebe, quantas doses de bebida alcoólica consome?

In [1271]:
print(df['P02801'].max()) # = 7 dias
print(df['P029'].max()) # = 60 doses

# → Máximo teórico de doses por semana = 7 * 60 = 420 doses

7.0
60.0


In [1272]:
def calcular_doses_semanais(row):
    freq = row['P027']
    dias_semana = row['P02801']
    doses_por_dia = row['P029']

    if freq == 1:
        return 0  # Não bebe nunca
    elif freq == 2:
        return doses_por_dia if pd.notna(doses_por_dia) else pd.NA
    elif freq == 3:
        if pd.notna(dias_semana) and pd.notna(doses_por_dia):
            return dias_semana * doses_por_dia
        else:
            return pd.NA
    return pd.NA

def categorizar_doses_por_sexo(row):
    doses = row['Doses_semanais']
    sexo = row['C006']

    if pd.isna(doses) or pd.isna(sexo):
        return pd.NA
    elif doses == 0:
        return 'Abstêmio'
    
    if sexo == 1:  # Homem
        if doses <= 7:
            return 'Baixo consumo'
        elif doses <= 14:
            return 'Moderado'
        elif doses <= 30:
            return 'Alto consumo'
        else:
            return 'Excessivo'
    elif sexo == 2:  # Mulher
        if doses <= 3:
            return 'Baixo consumo'
        elif doses <= 7:
            return 'Moderado'
        elif doses <= 15:
            return 'Alto consumo'
        else:
            return 'Excessivo'
    else:
        return pd.NA

# Aplicar ao DataFrame
df['Doses_semanais'] = df.apply(calcular_doses_semanais, axis=1)
df['Consumo_alcool'] = df.apply(categorizar_doses_por_sexo, axis=1)

print(df['Consumo_alcool'].value_counts(dropna=False))
print("Valores faltantes:", df['Consumo_alcool'].isna().sum())


Consumo_alcool
Abstêmio         13878
Baixo consumo     3732
Moderado          1584
Alto consumo       924
Excessivo          408
Name: count, dtype: int64
Valores faltantes: 0


In [1273]:
df[["Consumo_alcool"]]

,Consumo_alcool
0,Abstêmio
1,Alto consumo
2,Moderado
3,Abstêmio
4,Abstêmio
...,...
20523,Moderado
20524,Abstêmio
20525,Baixo consumo
20526,Alto consumo


In [1274]:
mapeamento = {
    "Abstêmio": 0,
    "Baixo consumo": 1,
    "Moderado": 2,
    "Alto consumo": 3,
    "Excessivo": 4
}

# Aplicando o mapeamento
df["Consumo_alcool"] = df["Consumo_alcool"].replace(mapeamento).astype(int)

In [1275]:
df[["Consumo_alcool"]]

,Consumo_alcool
0,0
1,3
2,2
3,0
4,0
...,...
20523,2
20524,0
20525,1
20526,3


In [1276]:
df = df.drop(columns=['P027','P02801', 'P029', 'Doses_semanais'])

### IMC

1. P00103: Peso - Informado (em kg)
2. P00104: Peso - Final (em kg)
3. P00403: Altura - Informada (em cm)
4. P00404: Altura - Final (em cm)

In [1277]:
print(df['P00103'].isna().sum())
print(df['P00104'].isna().sum())

1479
27


In [1278]:
print(df['P00403'].isna().sum())
print(df['P00404'].isna().sum())

4079
27


In [1279]:
faltantes_iguais = df[(df['P00104'].isna()) & (df['P00103'].notna())][['P00104', 'P00103']]
faltantes_iguais

,P00104,P00103


In [1280]:
faltantes_p00104 = df[df['P00104'].isna()][['P00103', 'P00104']]
faltantes_p00104

,P00103,P00104
416,NaN,NaN
1459,NaN,NaN
1483,NaN,NaN
1523,NaN,NaN
4388,NaN,NaN
4457,NaN,NaN
5426,NaN,NaN
6171,NaN,NaN
6574,NaN,NaN
7588,NaN,NaN


In [1281]:
faltantes_iguais = df[(df['P00404'].isna()) & (df['P00403'].notna())][['P00404', 'P00403']]
faltantes_iguais

,P00404,P00403


In [1282]:
faltantes_p00404 = df[df['P00404'].isna()][['P00403', 'P00404']]
faltantes_p00404

,P00403,P00404
416,NaN,NaN
1459,NaN,NaN
1483,NaN,NaN
1523,NaN,NaN
4388,NaN,NaN
4457,NaN,NaN
5426,NaN,NaN
6171,NaN,NaN
6574,NaN,NaN
7588,NaN,NaN


In [1283]:
faltantes = df['P00104'].isna().equals(df['P00404'].isna())
faltantes

True

In [1284]:
df = df.dropna(subset=['P00104', 'P00404'])

In [1285]:
print(df['P00104'].isna().sum())
print(df['P00404'].isna().sum())

0
0


In [1286]:
df = df.drop(columns=['P00103', 'P00403'])

In [1287]:
# Calculando o IMC (peso / altura²) e convertendo altura de cm para metros
df['imc'] = df['P00104'] / ((df['P00404'] / 100) ** 2)

# Classificando em faixas
conditions = [
    (df['imc'] < 18.5),
    ((df['imc'] >= 18.5) & (df['imc'] < 25)),
    ((df['imc'] >= 25) & (df['imc'] < 30)),
    ((df['imc'] >= 30) & (df['imc'] < 35)),
    ((df['imc'] >= 35) & (df['imc'] < 40)),
    (df['imc'] >= 40)
]

labels = [
    'Abaixo do peso',
    'Peso normal',
    'Sobrepeso',
    'Obesidade Grau I',
    'Obesidade Grau II',
    'Obesidade Grau III'
]

df['Faixa_imc'] = np.select(conditions, labels, default='Dados inválidos')

print(df['Faixa_imc'].value_counts(dropna=False))
print("Valores faltantes:", df['Faixa_imc'].isna().sum())

Faixa_imc
Sobrepeso             8016
Peso normal           7532
Obesidade Grau I      3463
Obesidade Grau II      904
Abaixo do peso         325
Obesidade Grau III     259
Name: count, dtype: int64
Valores faltantes: 0


Juntando obesidade pelo alto desbalanceamento entre categorias

In [1288]:
# Mapeamento explícito
mapeamento = {
    "Abaixo do peso": 0,
    "Peso normal": 1,
    "Sobrepeso": 2,
    "Obesidade Grau I": 3,
    "Obesidade Grau II": 3,
    "Obesidade Grau III": 3
}

# Aplicando o mapeamento
df["Faixa_imc"] = df["Faixa_imc"].replace(mapeamento).astype(int)

# Visualizando resultado
df[["Faixa_imc"]]

,Faixa_imc
0,3
1,3
2,2
3,2
4,3
...,...
20523,1
20524,1
20525,1
20526,2


In [1289]:
df = df.drop(columns=['imc']) 

### Juntando J007 e J00801

1. J007 -  Algum médico já deu o diagnóstico de alguma doença  crônica, física ou mental, ou doença de longa duração (de mais de 6 meses de duração) a  ___ ?
2. J00801 - Alguma dessas doenças limita de alguma forma suas atividades habituais (trabalhar, ir à escola, brincar, afazeres domésticos, etc.)

se J007 = 2, então 0.

se J007 = 1 e J00801 = 1, então 1.

se J007 = 1 e J00801 = 2, então 2.

In [1290]:
condicoes = [
    df['J007'] == 2,
    (df['J007'] == 1) & (df['J00801'] == 1),
    (df['J007'] == 1) & (df['J00801'] == 2)
]

# Valores correspondentes às condições
valores = [0, 1, 2]

# Criando novo atributo
df['Doenca_limitante'] = np.select(condicoes, valores, default=np.nan)

In [1291]:
print(df['Doenca_limitante'].isna().sum())

0


In [1292]:
df = df = df.drop(columns=['J007', 'J00801'])

### Lalala

In [1293]:
print("M00203 faltantes:", df['M00203'].isna().sum())
print(df['M00203'].value_counts(dropna=False)) #ver isso aqui (incompleto)

M00203 faltantes: 2
M00203
1.0    20305
2.0      189
9.0        3
NaN        2
Name: count, dtype: int64


In [1294]:
df = df[~(df['M00203'].isna() | (df['M00203'] == 9))]

In [1295]:
print(df['M00203'].value_counts(dropna=False))

M00203
1.0    20305
2.0      189
Name: count, dtype: int64


In [1296]:
df = df.drop(columns=['R034', 'E006011', 'E008', 'J002', 'J00402', 'J00404', 'C00301']) #incompleto (R034)

In [1297]:
df.to_csv('../Bases/Criados/4_base_preenchida.csv', index=False)

In [1298]:
df.columns[df.isnull().any()]

Index([], dtype='object')

In [1299]:
(df.isnull().sum() > 0).sum()

np.int64(0)

df = df.drop(columns=['B001', 'E006011', 'E008', 'J01101', 'M00203', 'P02001', 'P02002','P02501', 'P02602', 'P06701', 'T001', 'H001', 'H002', 'H003', 'H004','H010', 'N00101',
                      'Temporariamente_afastado', 'Mobilidade_auxiliar', 'Exposicao_quimica', 'Exposicao_ruido', 'Exposicao_lixo', 'Exposicao_minerio', 'Doses_semanais',
                      'Plano_publico', 'Cobertura_plano', 'P068'])